# DSE HA-3
# **Creating your own Sorting Hogwarts Hat ✨**

The aim of this task is to constuct ML model that predicts  Hogwarts Houses based on students forces (the target variable is `Hogwarts House`)

Good luck, dear wizards!

In [ ]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/Shuaynat/DSE-23-24/main/02-home-assignments/ha-03-practice/ha-3_dataset.csv")

In [ ]:
data.head()

## 1 Data preparation

Follow several preparation steps:

1. Drop columns that we will not use for prediction: last name, first name and index.
2. Have a look at the correlation plot. If you see that some features are too correlated - drop some of them
3. Fill mising features with mean taret encoding
4. Convert categorical variables `Best Hand` and `Hogwart House` to numerical.
5. Create from `Birthday` new fearures `day`, `month`, `year`
6. Standardize features
7. Separate data into test and train sets (leave 33% of data in test set)


In [ ]:
print(data.columns)
data.info()

In [ ]:
# 1-dropping columns
data.drop(columns=['Index','First Name','Last Name'],inplace=True)

In [ ]:
# 2-correlation

import numpy as np
corr_matrix = data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# correlation greater than 0.8
dropping_criteria = [column for column in upper.columns if any(upper[column] > 0.8)]

# Drop the features
data.drop(dropping_criteria, axis=1, inplace=True)

<ipython-input-4-60d30d7c37d5>:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = data.corr().abs()


In [ ]:
# 3-Filling missing features mean
import numpy as np

# Finding the mean of the column that have NaN values and replacing with the mean
mean_value = data['Arithmancy'].mean()
data['Arithmancy'].fillna(value=mean_value, inplace=True)
mean_value = data['Astronomy'].mean()
data['Astronomy'].fillna(value=mean_value, inplace=True)
mean_value = data['Herbology'].mean()
data['Herbology'].fillna(value=mean_value, inplace=True)
mean_value = data['Divination'].mean()
data['Divination'].fillna(value=mean_value, inplace=True)
mean_value = data['Muggle Studies'].mean()
data['Muggle Studies'].fillna(value=mean_value, inplace=True)
mean_value = data['Ancient Runes'].mean()
data['Ancient Runes'].fillna(value=mean_value, inplace=True)
mean_value = data['History of Magic'].mean()
data['History of Magic'].fillna(value=mean_value, inplace=True)
mean_value = data['Potions'].mean()
data['Potions'].fillna(value=mean_value, inplace=True)
mean_value = data['Care of Magical Creatures'].mean()
data['Care of Magical Creatures'].fillna(value=mean_value, inplace=True)

In [ ]:
# 4-Converting categorical variables
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Best Hand'] = le.fit_transform(data['Best Hand'])
data['Hogwarts House'] = le.fit_transform(data['Hogwarts House'])

In [ ]:
# 5-from Birthday new fearures day, month, year
import datetime
import pandas as pd
data["Birthday"] = pd.to_datetime(data["Birthday"])

data['Day']=data["Birthday"].dt.day
data['Month']=data["Birthday"].dt.month
data['Year']=data["Birthday"].dt.year

data.drop(columns=['Birthday'],inplace=True)

In [ ]:
y_ = data['Hogwarts House']
X_ = data.drop(['Hogwarts House'], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
# Initialize a scaler
scaler = StandardScaler()
# Fit and transform the data
X = scaler.fit_transform(X_)
y = y_

In [ ]:
# 7-Separate data into test and train sets (leave 33% of data in test set)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,train_size = 0.66)

## 2. Multiclass logit model with regularization


Estimate multiclass logit model:

1. Use sklearn realization of logistic regression in [SGDC classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)


2. Use cross-validation to choose type of regularization (penalty equal to `l1`, `l2`, or `elastic net`) and regularization weight (`alpha` in  `range(start=0.1, 1, step=0.1)`)


In [ ]:
def evaluate(model, x_train, y_train, x_test, y_test):
    train_preds = model.predict(x_train)
    test_preds = model.predict(x_test)
    train_acc = metrics.accuracy_score(y_train, train_preds)
    test_acc = metrics.accuracy_score(y_test, test_preds)
    print('Train accuracy: %s' % train_acc)
    print('Test accuracy: %s' % test_acc)

In [ ]:
from sklearn import linear_model, metrics, model_selection
import numpy as np
grid_search = model_selection.GridSearchCV(
    estimator=linear_model.SGDClassifier(),
    param_grid={'alpha': np.arange(0.1, 1, 0.1),
                'penalty':['l2','l1','elasticnet']},
    cv=10,
    return_train_score=True)
grid_search.fit(X_train, y_train)

r = pd.DataFrame(grid_search.cv_results_)
print(r[['params', 'mean_train_score', 'mean_test_score']])

best_alpha = grid_search.best_params_['alpha']
best_penalty = grid_search.best_params_['penalty']
print('Best alpha: %s' % best_alpha)
print('Best penalty: %s' % best_penalty)

In [ ]:
best_model = linear_model.SGDClassifier(alpha=best_alpha, penalty=best_penalty)
best_model.fit(X_train, y_train)
evaluate(best_model, X_train, y_train, X_test, y_test)

Train accuracy: 0.9801136363636364
Test accuracy: 0.9852941176470589


In [ ]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Always scale the input. The most convenient way is to use a pipeline.
logit_model = SGDClassifier(penalty='l2',alpha=0.1)
logit_model.fit(X_train, y_train)

from sklearn.metrics import classification_report, confusion_matrix
y_predict_logit_model = logit_model.predict(X_test)

## 3. Calculate average marginal effects




For each of the variables calculate average marginal effect on the probability.

In [ ]:
def logit_margeff(model, X, features, kind='probability'):

    coef = model.coef_
    intercept = model.intercept_

    if kind == 'probability':

        logodds = intercept + X @ coef.T

        marg_effects=[]
        for i in range(coef.size):
            marg_eff = np.mean(coef[0,i]*np.exp(-logodds)/(1+np.exp(-logodds))**2).round(3)[0]
            marg_effects.append(marg_eff)


    elif kind == "odds":

        marg_effects=[]
        for i in range(coef.size):
            marg_eff = (np.exp(coef[0,i])).round(3)
            marg_effects.append(marg_eff)

    marginal_effects = {}
    marginal_effects['features'] = features
    marginal_effects[f'marginal_effects_{kind}'] = marg_effects

    df = pd.DataFrame(marginal_effects)

    return df

In [ ]:
features = list(data.iloc[:,1:13].columns)
print(features)

logit_model = SGDClassifier(penalty='l2',alpha=0.1)
logit_model.fit(data[features], data['Hogwarts House'])


logit_margeff(logit_model,data[features],features,kind='probability')


## 4. Build logit with PCA transformation


Extract 5 principal components and evaluate logit model using them as explanatory variables

In [ ]:
# YOUR CODE HERE
from sklearn.decomposition import PCA

pca = PCA(n_components=5).fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

# Always scale the input. The most convenient way is to use a pipeline.
logitpca_model = SGDClassifier(penalty='l2',alpha=0.1)
logitpca_model.fit(X_train, y_train)

y_predict_logitpca_model = logitpca_model.predict(X_test)

## 5. Build classification forest


You are free to add PCA-transformed explanatory variables as predictors. Tune parameters by cross-validation on the grid of your choice

In [ ]:
from sklearn import linear_model, metrics, model_selection
from sklearn.ensemble import RandomForestClassifier
grid_search = model_selection.GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid={'max_depth':[1, 3, 5, 10],
                'criterion':['gini','entropy','log_loss']},
    cv=10,
    return_train_score=True)
grid_search.fit(X_train, y_train)

r = pd.DataFrame(grid_search.cv_results_)
print(r[['params', 'mean_train_score', 'mean_test_score']])

best_max_depth = grid_search.best_params_['max_depth']
best_criterion = grid_search.best_params_['criterion']
print('Best max_depth: %s' % best_max_depth)
print('Best criterion: %s' % best_criterion)

                                        params  mean_train_score  \
0        {'criterion': 'gini', 'max_depth': 1}          0.866807   
1        {'criterion': 'gini', 'max_depth': 3}          0.978641   
2        {'criterion': 'gini', 'max_depth': 5}          0.981586   
3       {'criterion': 'gini', 'max_depth': 10}          0.999053   
4     {'criterion': 'entropy', 'max_depth': 1}          0.881207   
5     {'criterion': 'entropy', 'max_depth': 3}          0.979482   
6     {'criterion': 'entropy', 'max_depth': 5}          0.980955   
7    {'criterion': 'entropy', 'max_depth': 10}          0.999684   
8    {'criterion': 'log_loss', 'max_depth': 1}          0.845438   
9    {'criterion': 'log_loss', 'max_depth': 3}          0.979587   
10   {'criterion': 'log_loss', 'max_depth': 5}          0.980744   
11  {'criterion': 'log_loss', 'max_depth': 10}          0.999684   

    mean_test_score  
0          0.868221  
1          0.977260  
2          0.978203  
3          0.978203  
4    

In [ ]:
best_model = RandomForestClassifier(max_depth=best_max_depth, criterion=best_criterion)
best_model.fit(X_train, y_train)
y_predict_rf_model = best_model.predict(X_test)
evaluate(best_model, X_train, y_train, X_test, y_test)

Train accuracy: 0.9801136363636364
Test accuracy: 0.9797794117647058


## 6. Model Race: evaluate models (Logit, Logit + PCA, Classification forest) on the test dataset and choose the best one

In [ ]:
# YOUR CODE HERE

cm_logit = confusion_matrix(y_test, y_predict_logit_model)
print(cm_logit)
cm_logitpca = confusion_matrix(y_test, y_predict_logitpca_model)
print(cm_logitpca)
cm_rf = confusion_matrix(y_test, y_predict_rf_model)
print(cm_rf)

# All models result in same performance

[[100   3   1   0]
 [  1 187   0   1]
 [  0   1 154   1]
 [  0   0   0  95]]
[[100   3   1   0]
 [  1 187   0   1]
 [  0   1 154   1]
 [  0   0   0  95]]
[[100   3   1   0]
 [  2 186   0   1]
 [  0   1 154   1]
 [  0   2   0  93]]


   



---

![Alt Text](https://gifdb.com/images/high/gryffindor-sorting-hat-owbd80ftf54oai7i.webp)


